## Configuração do ambiente para utilização do Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# instalando a findspark
!pip install -q findspark

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Minha Primeira Aplicação no Pyspark") \
        .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Leitura dos dados

In [2]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/bases_dados_projeto_credito/previous_application.csv')

In [3]:
# Função para substituir espaço por underline e colocar a palavra em letras maiúsculas
import re
def tratamento_palavra(palavra):
    palavra_transformada = re.sub(r'\s', '_', palavra).upper()
    return palavra_transformada

In [4]:
status = df1.NAME_CONTRACT_STATUS.unique()
nova_lista = []

for palavra in status:
  nova_lista.append(tratamento_palavra(palavra))

In [5]:
# Lê o arquivo Parquet
dados = spark.read.csv("/content/drive/MyDrive/bases_dados_projeto_credito/POS_CASH_balance.csv",header=True)

# Mostra os dados
dados.show()

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|   1803195|    182943|           -31|          48.0|                 45.0|              Active|     0|         0|
|   1715348|    367990|           -33|          36.0|                 35.0|              Active|     0|         0|
|   1784872|    397406|           -32|          12.0|                  9.0|              Active|     0|         0|
|   1903291|    269225|           -35|          48.0|                 42.0|              Active|     0|         0|
|   2341044|    334279|           -35|          36.0|                 35.0|              Active|     0|         0|
|   2207092|    342166|           -32|          12.0|                 12.0|     

In [6]:
# Extraia os valores únicos da coluna 'NAME_CONTRACT_STATUS'
status = dados.select("NAME_CONTRACT_STATUS").distinct().rdd.flatMap(lambda x: x).collect()

# Defina a função de tratamento de palavra
def tratamento_palavra(palavra):
    palavra_transformada = re.sub(r'\s', '_', palavra).upper()
    return palavra_transformada

# Aplique a função de tratamento de palavra a cada valor único
nova_lista = [tratamento_palavra(status) for status in status]

# Exiba a lista resultante
print(nova_lista)

['DEMAND', 'APPROVED', 'COMPLETED', 'AMORTIZED_DEBT', 'RETURNED_TO_THE_STORE', 'XNA', 'ACTIVE', 'SIGNED', 'CANCELED']


## Criação de flags para nos auxiliar na visão temporal dos dados

In [7]:
## Habilitando uso do SparkSQL
dados.createOrReplaceTempView("dados")

df_temp_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN MONTHS_BALANCE >= -3 THEN 1
        ELSE 0
    END AS ultimos_3_meses,
    CASE
        WHEN MONTHS_BALANCE >= -6 THEN 1
        ELSE 0
    END AS ultimos_6_meses,
    CASE
        WHEN MONTHS_BALANCE >= -12 THEN 1
        ELSE 0
    END AS ultimos_12_meses,
    CASE
        WHEN MONTHS_BALANCE >= -24 THEN 1
        ELSE 0
    END AS ultimos_24_meses,
    CASE
        WHEN MONTHS_BALANCE >= -36 THEN 1
        ELSE 0
    END AS ultimos_36_meses
FROM dados
ORDER BY `SK_ID_PREV`;
""")
df_temp_01.createOrReplaceTempView("df_temp_01")
df_temp_01.show()

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+---------------+---------------+----------------+----------------+----------------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|ultimos_3_meses|ultimos_6_meses|ultimos_12_meses|ultimos_24_meses|ultimos_36_meses|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+---------------+---------------+----------------+----------------+----------------+
|   1000001|    158271|            -8|           2.0|                  0.0|           Completed|     0|         0|              0|              0|               1|               1|               1|
|   1000001|    158271|            -9|          12.0|                 11.0|              Active|     0|         0|              0|              0|               1|               1|               1|
|   100000

## Sumarizar na visão cliente (Automatizada)

In [8]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when

# Definir as colunas para agregação
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

colunas_flags = ['ultimos_3_meses','ultimos_6_meses','ultimos_12_meses','ultimos_24_meses','ultimos_36_meses']

expressoes_agregacao = []

for flag in colunas_flags:
  for coluna in colunas_agregacao_total:
    if 'DPD' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_POS_CASH"))
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_POS_CASH"))
    else:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_POS_CASH"))
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_POS_CASH"))
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_POS_CASH"))
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_POS_CASH"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_02 = df_temp_01.groupBy("SK_ID_PREV").agg(*expressoes_agregacao).orderBy("SK_ID_PREV")

# Mostrar o DataFrame resultante
df_temp_02.show()

+----------+----------------------------------------------+----------------------------------------------+----------------------------------------------+----------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+--------------------------------------+--------------------------------------+------------------------------------------+------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+------------------------------------

In [9]:
# Definir as colunas para agregação
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

colunas_flags = ['ultimos_3_meses','ultimos_6_meses','ultimos_12_meses','ultimos_24_meses','ultimos_36_meses']

expressoes_agregacao = []

for categoria in nova_lista:
  for flag in colunas_flags:
    for coluna in colunas_agregacao_total:
      if 'DPD' in coluna:
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
      else:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_03 = df_temp_01.groupBy("SK_ID_PREV").agg(*expressoes_agregacao).orderBy("SK_ID_PREV")

# Mostrar o DataFrame resultante
df_temp_03.show()

+----------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+---------------------------------------------+---------------------------------------------+-------------------------------------------------+-------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------

## Join das duas bases

In [10]:
df_temp_04 = df_temp_02.join(df_temp_03, "SK_ID_PREV")

## Salvar a tabela sumarizada

In [11]:
df_temp_04 = df_temp_02.repartition(1)
df_temp_04.write.mode("overwrite").csv("pos_cash_agg.csv",header=True)